In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest, DateRange, Dimension, Metric
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import json, os

In [3]:
SCOPES = ["https://www.googleapis.com/auth/analytics.readonly"]
CLIENT_SECRETS = "credenciais.json"   # seu OAuth “installed” JSON
TOKEN_FILE    = "token.json"
PROPERTY_ID   = "213025502"           # ID certo da Demo GA4

def get_client():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as f:
            f.write(creds.to_json())
    return BetaAnalyticsDataClient(credentials=creds)

client = get_client()
request = RunReportRequest(
    property=f"properties/{PROPERTY_ID}",
    dimensions=[Dimension(name="date")],
    metrics=[Metric(name="activeUsers")],
    date_ranges=[DateRange(start_date="2023-12-01", end_date="2023-12-07")],
)
resp = client.run_report(request)
for row in resp.rows:
    print(row.dimension_values[0].value, row.metric_values[0].value)

ResourceExhausted: 429 This property is denied access to the API. Contact Google Analytics API Support for more information.